In [7]:
# #!/usr/bin/env python3
# %pip install numpy scikit-learn opencv-python-headless
# """
# A script to process images for multiple classes from specific folders.

# How it works:
# 1.  Loops through a list of predefined class names.
# 2.  For each class, reads images from 'defect' and 'non-defect' subfolders.
# 3.  Assigns label '1' to defect images and '0' to non-defect images.
# 4.  Resizes and normalizes all images for that class.
# 5.  Splits the data into training and validation sets.
# 6.  Saves the processed data into class-specific subdirectories.
# """
# import os
# import cv2
# import numpy as np
# import pickle
# from glob import glob
# from sklearn.model_selection import train_test_split
# from typing import Tuple, List, Dict, Any, Optional

# # ===============================================================
# #                       CONFIGURATION
# #          --->>> EDIT THESE VARIABLES <<<---
# # ===============================================================

# # 1. List of class subdirectories to process.
# CLASS_NAMES = ["lighting_panel", "shifted_grab_handle", "frosted_window"]

# # 2. Path where the processed data will be saved.
# OUTPUT_DIR = 'processed_data_multi_class/'

# # 3. The size to resize all images to (height, width).
# IMAGE_SIZE = (224, 224)

# # 4. The percentage of data to use for validation (e.g., 0.2 = 20%).
# VAL_SPLIT = 0.3

# # ===============================================================


# def load_and_preprocess_data_for_class(class_name: str) -> Tuple[Optional[Tuple[np.ndarray, np.ndarray]], Optional[Tuple[np.ndarray, np.ndarray]]]:
#     """
#     Loads images for a single class, preprocesses them,
#     and splits into train/validation sets.
#     """
#     images: List[np.ndarray] = []
#     labels: List[int] = []

#     # Define the paths for the current class
#     path_non_defect = f'Dataset/AR_Train/non-defect/{class_name}'
#     path_defect = f'Dataset/AR_Train/defect/{class_name}'
    
#     classes_to_process = {
#         'non-defect': {'path': path_non_defect, 'label': 0},
#         'defect': {'path': path_defect, 'label': 1}
#     }

#     print(f"--- Processing class: {class_name} ---")
#     for sub_class, info in classes_to_process.items():
#         folder_path = info['path']
#         label = info['label']
        
#         if not os.path.isdir(folder_path):
#             print(f"  WARNING: Directory not found, skipping: {folder_path}")
#             continue

#         image_paths = glob(os.path.join(folder_path, '*.jpg'))
        
#         print(f"  Found {len(image_paths)} images for '{sub_class}'")

#         for path in image_paths:
#             try:
#                 img = cv2.imread(path)
#                 if img is None:
#                     print(f"  WARNING: Could not read image, skipping: {path}")
#                     continue
                
#                 # Resize and normalize
#                 img_resized = cv2.resize(img, (IMAGE_SIZE[1], IMAGE_SIZE[0]))
#                 img_normalized = img_resized.astype(np.float32) / 255.0
                
#                 images.append(img_normalized)
#                 labels.append(label)
#             except Exception as e:
#                 print(f"  ERROR: Failed processing {path}: {e}")

#     if not images:
#         print(f"  ERROR: No images were loaded for class '{class_name}'. Aborting this class.")
#         return (None, None), (None, None)

#     X = np.array(images)
#     y = np.array(labels)

#     # Split data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(
#         X, y, test_size=VAL_SPLIT, random_state=42, stratify=y
#     )
    
#     print(f"  Data split complete. Training: {len(X_train)}, Validation: {len(X_val)}")
#     return (X_train, y_train), (X_val, y_val)

# def save_data(data: Dict[str, Any], output_path: str, filename: str) -> None:
#     """Saves data to a file using pickle."""
#     os.makedirs(output_path, exist_ok=True)
#     filepath = os.path.join(output_path, filename)
#     with open(filepath, 'wb') as f:
#         pickle.dump(data, f)
#     print(f"  Data saved to: {filepath}")

# def main():
#     """Main execution function to loop through all classes."""
#     for class_name in CLASS_NAMES:
#         # Create class-specific output directories
#         output_class_dir = os.path.join(OUTPUT_DIR, class_name)
#         output_train_dir = os.path.join(output_class_dir, 'train')
#         output_val_dir = os.path.join(output_class_dir, 'val')

#         # Load and process data for the current class
#         (X_train, y_train), (X_val, y_val) = load_and_preprocess_data_for_class(class_name)
        
#         if X_train is not None and y_train is not None:
#             save_data({'images': X_train, 'labels': y_train}, output_train_dir, 'train_data.pkl')
#             save_data({'images': X_val, 'labels': y_val}, output_val_dir, 'val_data.pkl')
#             print(f"--- Finished processing for class: {class_name} ---\n")
#         else:
#             print(f"--- Failed to process class: {class_name} ---\n")

# if __name__ == "__main__":
#     main()

In [8]:
#!/usr/bin/env python3
%pip install numpy scikit-learn opencv-python-headless
"""
A script to process images for multiple classes and prepare
data for 3-fold cross-validation.

How it works:
1.  Loops through a list of predefined class names.
2.  For each class, reads all 'defect' (label 1) and 'non-defect' (label 0) images.
3.  Resizes and normalizes all images.
4.  Uses StratifiedKFold to split the data into 3 folds for cross-validation.
5.  Each fold consists of a training set (~67%) and a validation set (~33%).
6.  Saves the processed data for each fold into class-specific subdirectories.
"""
import os
import cv2
import numpy as np
import pickle
from glob import glob
from sklearn.model_selection import StratifiedKFold
from typing import Tuple, List, Dict, Any, Optional

# ===============================================================
#                       CONFIGURATION
#             --->>> EDIT THESE VARIABLES <<<---
# ===============================================================

# 1. List of class subdirectories to process.
CLASS_NAMES = ["lighting_panel", "shifted_grab_handle", "frosted_window"]

# 2. Path where the processed data will be saved.
OUTPUT_DIR = 'processed_data_multi_class_3_fold/'

# 3. The size to resize all images to (height, width).
IMAGE_SIZE = (224, 224)

# 4. The number of folds for cross-validation.
N_FOLDS = 3

# ===============================================================


def load_all_data_for_class(class_name: str) -> Optional[Tuple[np.ndarray, np.ndarray]]:
    """
    Loads and preprocesses all images for a single class.
    """
    images: List[np.ndarray] = []
    labels: List[int] = []

    # Define the paths for the current class
    path_non_defect = f'Dataset/AR_Train/non-defect/{class_name}'
    path_defect = f'Dataset/AR_Train/defect/{class_name}'
    
    classes_to_process = {
        'non-defect': {'path': path_non_defect, 'label': 0},
        'defect': {'path': path_defect, 'label': 1}
    }

    print(f"--- Loading data for class: {class_name} ---")
    for sub_class, info in classes_to_process.items():
        folder_path = info['path']
        label = info['label']
        
        if not os.path.isdir(folder_path):
            print(f"  WARNING: Directory not found, skipping: {folder_path}")
            continue

        image_paths = glob(os.path.join(folder_path, '*.jpg'))
        
        print(f"  Found {len(image_paths)} images for '{sub_class}'")

        for path in image_paths:
            try:
                img = cv2.imread(path)
                if img is None:
                    print(f"  WARNING: Could not read image, skipping: {path}")
                    continue
                
                # Resize and normalize
                img_resized = cv2.resize(img, (IMAGE_SIZE[1], IMAGE_SIZE[0]))
                img_normalized = img_resized.astype(np.float32) / 255.0
                
                images.append(img_normalized)
                labels.append(label)
            except Exception as e:
                print(f"  ERROR: Failed processing {path}: {e}")

    if not images:
        print(f"  ERROR: No images were loaded for class '{class_name}'. Aborting this class.")
        return None

    X = np.array(images)
    y = np.array(labels)
    
    print(f"  Successfully loaded {len(X)} total images for this class.")
    return X, y

def save_data(data: Dict[str, Any], output_path: str, filename: str) -> None:
    """Saves data to a file using pickle."""
    os.makedirs(output_path, exist_ok=True)
    filepath = os.path.join(output_path, filename)
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"  Data saved to: {filepath}")

def main():
    """Main execution function to loop through all classes and create k-fold splits."""
    for class_name in CLASS_NAMES:
        # Load all data for the current class
        data = load_all_data_for_class(class_name)
        
        if data is None:
            print(f"--- Failed to process class: {class_name} ---\n")
            continue
            
        X, y = data
        
        # Initialize Stratified K-Fold
        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

        # Loop through each fold
        print(f"\n--- Creating {N_FOLDS}-fold splits for class: {class_name} ---")
        for fold_idx, (train_indices, val_indices) in enumerate(skf.split(X, y)):
            fold_num = fold_idx + 1
            print(f"  Processing Fold {fold_num}/{N_FOLDS}...")
            
            # Create data splits for the current fold
            X_train, X_val = X[train_indices], X[val_indices]
            y_train, y_val = y[train_indices], y[val_indices]
            
            print(f"    Training set size: {len(X_train)}")
            print(f"    Validation set size: {len(X_val)}")

            # Define output directory for the current fold
            output_fold_dir = os.path.join(OUTPUT_DIR, class_name, f'fold_{fold_num}')
            
            # Save the training and validation data for the fold
            save_data({'images': X_train, 'labels': y_train}, output_fold_dir, 'train_data.pkl')
            save_data({'images': X_val, 'labels': y_val}, output_fold_dir, 'val_data.pkl')
            
        print(f"--- Finished processing for class: {class_name} ---\n")

if __name__ == "__main__":
    main()

  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (54.0 MB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python-headless]v-python-headless]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires onnxruntime<2,>=1.15.0, which is not installed.
autogluon-multimodal 1.4.0 requires nvidia-ml-py

In [1]:
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg16, VGG16_Weights
from torchvision import transforms
import numpy as np

# For timestamping
from datetime import datetime
import pytz

# Set timezone for Lahore, Pakistan
pk_timezone = pytz.timezone("Asia/Karachi")

In [8]:
class Config:
    # --- Paths ---
    # The base directory where all class folders are located
    base_data_dir = 'processed_data_multi_class_3_fold/'
    # The base directory where all trained models will be saved
    base_output_dir = 'trained_models_multi_class/'

    # --- Classes to Train ---
    # The script will train one model for each class in this list.
    CLASS_NAMES = ["lighting_panel", "shifted_grab_handle", "frosted_window"]

    # --- Model Parameters ---
    # This is a binary classification (defect vs. non-defect) for each class
    num_classes = 2
    N_FOLDS = 3

    # --- ADL Specific Parameters ---
    drop_rate = 0.4
    drop_threshold = 0.8
    adl_alpha = 0.1

    # --- Training Hyperparameters ---
    epochs = 50
    batch_size = 16
    lr = 0.001
    num_workers = 2

# Instantiate the config object
config = Config()

# Create the base output directory if it doesn't exist
os.makedirs(config.base_output_dir, exist_ok=True)

In [9]:
class ADL(nn.Module):
    def __init__(self, in_channels: int, drop_rate: float, drop_threshold: float):
        super().__init__()
        if not (0.0 <= drop_rate < 1.0):
            raise ValueError("drop_rate must be in the range [0, 1)")
        if not (0.0 <= drop_threshold < 1.0):
            raise ValueError("drop_threshold must be in the range [0, 1)")

        self.drop_rate = drop_rate
        self.drop_threshold = drop_threshold
        self.attention_conv = nn.Conv2d(in_channels, 1, kernel_size=1)

    def forward(self, feature_maps: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        attention_map = self.attention_conv(feature_maps)
        attention_map = torch.sigmoid(attention_map)

        if self.training:
            B, _, H, W = attention_map.shape
            attention_flat = attention_map.view(B, -1)
            M = int(H * W * self.drop_rate)
            if M <= 0:
                return feature_maps, attention_map
            _, topk_indices = torch.topk(attention_flat, k=M, dim=1)
            drop_mask = torch.ones_like(attention_flat)
            drop_mask.scatter_(dim=1, index=topk_indices, value=0)
            drop_mask = drop_mask.view(B, 1, H, W)
        else:
            drop_mask = (attention_map < self.drop_threshold).float()

        dropped_feature_maps = feature_maps * drop_mask
        return dropped_feature_maps, attention_map

In [10]:
class PreprocessedDataset(Dataset):
    def __init__(self, pkl_path: str):
        print(f"Loading data from {pkl_path}...")
        try:
            with open(pkl_path, 'rb') as f:
                self.data = pickle.load(f)
        except FileNotFoundError:
            print(f"ERROR: Data file not found at {pkl_path}")
            raise
        self.images = self.data['images']
        self.labels = self.data['labels']
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        print("Data loaded successfully.")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_bgr = self.images[idx]
        label = self.labels[idx]
        image_rgb = image_bgr[..., ::-1].copy() # BGR to RGB
        image_tensor = self.transform(image_rgb)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return image_tensor, label_tensor

In [11]:
class ADLModel(nn.Module):
    def __init__(self, num_classes: int, drop_rate: float, drop_threshold: float):
        super().__init__()
        vgg_full = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        self.backbone = vgg_full.features
        in_channels = 512
        self.adl_layer = ADL(in_channels, drop_rate, drop_threshold)
        self.classifier = nn.Conv2d(in_channels, num_classes, kernel_size=1)

    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        feature_maps = self.backbone(x)
        dropped_maps, attention_map = self.adl_layer(feature_maps)
        logits = self.classifier(dropped_maps)
        logits = F.adaptive_avg_pool2d(logits, (1, 1))
        logits = logits.squeeze(-1).squeeze(-1)
        return logits, attention_map

In [12]:
# def train_one_epoch(model, dataloader, criterion_cls, optimizer, device, adl_alpha):
#     model.train()
#     running_loss = 0.0
#     running_loss_cls = 0.0
#     running_loss_adl = 0.0
#     correct_predictions = 0
#     total_samples = 0
#     for inputs, labels in dataloader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         logits, attention_map = model(inputs)
#         loss_cls = criterion_cls(logits, labels)
#         loss_adl = torch.mean(attention_map)
#         total_loss = loss_cls + adl_alpha * loss_adl
#         total_loss.backward()
#         optimizer.step()
#         running_loss += total_loss.item() * inputs.size(0)
#         running_loss_cls += loss_cls.item() * inputs.size(0)
#         running_loss_adl += loss_adl.item() * inputs.size(0)
#         _, preds = torch.max(logits, 1)
#         correct_predictions += torch.sum(preds == labels.data)
#         total_samples += labels.size(0)
#     epoch_loss = running_loss / total_samples
#     epoch_acc = correct_predictions.double() / total_samples
#     epoch_loss_cls = running_loss_cls / total_samples
#     epoch_loss_adl = running_loss_adl / total_samples
#     return epoch_loss, epoch_acc, epoch_loss_cls, epoch_loss_adl
def train_one_epoch(model, dataloader, criterion_cls, optimizer, device, adl_alpha):
    model.train()
    running_loss = 0.0
    running_loss_cls = 0.0
    running_loss_adl = 0.0
    correct_predictions = 0
    total_samples = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logits, attention_map = model(inputs)
        
        loss_cls = criterion_cls(logits, labels)
        loss_adl = torch.mean(attention_map)
        total_loss = loss_cls + adl_alpha * loss_adl
        
        total_loss.backward()
        optimizer.step()
        
        running_loss += total_loss.item() * inputs.size(0)
        running_loss_cls += loss_cls.item() * inputs.size(0)
        running_loss_adl += loss_adl.item() * inputs.size(0)
        
        _, preds = torch.max(logits, 1)
        correct_predictions += torch.sum(preds == labels.data)
        total_samples += labels.size(0)
        
    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions.double() / total_samples
    epoch_loss_cls = running_loss_cls / total_samples
    epoch_loss_adl = running_loss_adl / total_samples
    
    # CORRECTED RETURN STATEMENT: Returns a dictionary instead of a tuple
    return {
        "loss": epoch_loss,
        "accuracy": epoch_acc.item(), # Use .item() to get a standard Python number
        "cls_loss": epoch_loss_cls,
        "adl_loss": epoch_loss_adl
    }

# def validate(model, dataloader, criterion_cls, device):
#     model.eval()
#     running_loss = 0.0
#     correct_predictions = 0
#     total_samples = 0
#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             logits, _ = model(inputs)
#             loss = criterion_cls(logits, labels)
#             running_loss += loss.item() * inputs.size(0)
#             _, preds = torch.max(logits, 1)
#             correct_predictions += torch.sum(preds == labels.data)
#             total_samples += labels.size(0)
#     epoch_loss = running_loss / total_samples
#     epoch_acc = correct_predictions.double() / total_samples
#     return epoch_loss, epoch_acc
def validate(model, dataloader, criterion_cls, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, _ = model(inputs)
            loss = criterion_cls(logits, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(logits, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Calculate metrics
    epoch_loss = running_loss / len(all_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    accuracy = np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)

    # CORRECTED RETURN STATEMENT: Now returns a dictionary
    return {
        "loss": epoch_loss,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "labels": all_labels,
        "predictions": all_preds
    }

In [13]:
# # --- Setup ---
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # --- Data Loading ---
# train_dataset = PreprocessedDataset(os.path.join(config.data_dir, 'train', 'train_data.pkl'))
# val_dataset = PreprocessedDataset(os.path.join(config.data_dir, 'val', 'val_data.pkl'))
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
# val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

# # --- Model, Loss, Optimizer ---
# model = ADLModel(
#     num_classes=config.num_classes,
#     drop_rate=config.drop_rate,
#     drop_threshold=config.drop_threshold
# ).to(device)

# criterion_cls = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=config.lr)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)

# # --- Training Loop ---
# best_val_acc = 0.0
# print("\nStarting training...")
# start_time_pk = datetime.now(pk_timezone)
# print(f"Start Time (Lahore): {start_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")

# for epoch in range(config.epochs):
#     print(f"\n--- Epoch {epoch+1}/{config.epochs} ---")
#     train_loss, train_acc, loss_cls, loss_adl = train_one_epoch(
#         model, train_loader, criterion_cls, optimizer, device, config.adl_alpha)
#     print(f"Train | Total Loss: {train_loss:.4f} | Cls Loss: {loss_cls:.4f} | ADL Loss: {loss_adl:.4f} | Acc: {train_acc:.4f}")

#     val_loss, val_acc = validate(model, val_loader, criterion_cls, device)
#     print(f"Val   | Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")
    
#     scheduler.step()

#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model.state_dict(), os.path.join(config.output_dir, 'best_model_adl.pth'))
#         print(f"New best model saved with val acc: {best_val_acc:.4f}")

# end_time_pk = datetime.now(pk_timezone)
# print(f"\n--- Training Finished ---")
# print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
# print(f"Total training duration: {end_time_pk - start_time_pk}")
# print(f"Best validation accuracy achieved: {best_val_acc:.4f}")

In [14]:
# # --- Setup ---
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Store results for each class
# results = {}

# # Loop through each class name defined in the config
# for class_name in config.CLASS_NAMES:
#     print(f"\n========================================================")
#     print(f"        STARTING TRAINING FOR CLASS: {class_name}")
#     print(f"========================================================")

#     # --- Create class-specific paths ---
#     current_data_dir = os.path.join(config.base_data_dir, class_name)
#     current_output_dir = os.path.join(config.base_output_dir, class_name)
#     os.makedirs(current_output_dir, exist_ok=True)

#     # --- Data Loading for the current class ---
#     train_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'train', 'train_data.pkl'))
#     val_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'val', 'val_data.pkl'))
#     train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
#     val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

#     # --- Model, Loss, Optimizer (re-initialized for each class) ---
#     model = ADLModel(
#         num_classes=config.num_classes,
#         drop_rate=config.drop_rate,
#         drop_threshold=config.drop_threshold
#     ).to(device)

#     criterion_cls = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=config.lr)
#     scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)

#     # --- Training Loop for the current class ---
#     best_val_acc = 0.0
#     start_time_pk = datetime.now(pk_timezone)
#     print(f"Start Time (Lahore): {start_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")

#     for epoch in range(config.epochs):
#         print(f"\n--- Epoch {epoch+1}/{config.epochs} ---")
#         train_loss, train_acc, loss_cls, loss_adl = train_one_epoch(
#             model, train_loader, criterion_cls, optimizer, device, config.adl_alpha)
#         print(f"Train | Total Loss: {train_loss:.4f} | Cls Loss: {loss_cls:.4f} | ADL Loss: {loss_adl:.4f} | Acc: {train_acc:.4f}")

#         val_loss, val_acc = validate(model, val_loader, criterion_cls, device)
#         print(f"Val   | Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

#         scheduler.step()

#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             torch.save(model.state_dict(), os.path.join(current_output_dir, 'best_model_adl.pth'))
#             print(f"New best model for '{class_name}' saved with val acc: {best_val_acc:.4f}")

#     end_time_pk = datetime.now(pk_timezone)
#     results[class_name] = best_val_acc
#     print(f"\n--- Training for '{class_name}' Finished ---")
#     print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
#     print(f"Total training duration: {end_time_pk - start_time_pk}")
#     print(f"Best validation accuracy for '{class_name}': {best_val_acc:.4f}")


# # --- Final Summary ---
# print("\n========================================================")
# print("              ALL TRAINING RUNS COMPLETE")
# print("========================================================")
# for class_name, acc in results.items():
#     print(f"  - Best validation accuracy for '{class_name}': {acc:.4f}")
# print("========================================================")

In [15]:
# --- Setup ---
# In Cell 1

# Make sure this line includes all four functions
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
results = {}

# --- Main Loop ---
# for class_name in config.CLASS_NAMES:
#     print(f"\n========================================================")
#     print(f"        STARTING TRAINING FOR CLASS: {class_name}")
#     print(f"========================================================")

#     # --- Paths and History Initialization ---
#     current_data_dir = os.path.join(config.base_data_dir, class_name)
#     current_output_dir = os.path.join(config.base_output_dir, class_name)
#     os.makedirs(current_output_dir, exist_ok=True)
    
#     history = {
#         'train_loss': [], 'train_acc': [],
#         'val_loss': [], 'val_acc': [],
#         'val_precision': [], 'val_recall': [], 'val_f1': []
#     }
#     best_val_metrics = None

#     # --- Data Loading ---
#     train_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'train', 'train_data.pkl'))
#     val_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'val', 'val_data.pkl'))
#     train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
#     val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

#     # --- Model, Loss, Optimizer ---
#     model = ADLModel(config.num_classes, config.drop_rate, config.drop_threshold).to(device)
#     criterion_cls = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=config.lr)
#     scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)

#     # --- Training Loop ---
#     best_val_acc = 0.0
#     start_time_pk = datetime.now(pk_timezone)
#     print(f"Start Time (Lahore): {start_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")

#     for epoch in range(config.epochs):
#         print(f"\n--- Epoch {epoch+1}/{config.epochs} ---")
        
#         train_metrics = train_one_epoch(model, train_loader, criterion_cls, optimizer, device, config.adl_alpha)
#         val_metrics = validate(model, val_loader, criterion_cls, device)
        
#         # Log metrics
#         history['train_loss'].append(train_metrics['loss'])
#         history['train_acc'].append(train_metrics['accuracy'])
#         history['val_loss'].append(val_metrics['loss'])
#         history['val_acc'].append(val_metrics['accuracy'])
#         history['val_precision'].append(val_metrics['precision'])
#         history['val_recall'].append(val_metrics['recall'])
#         history['val_f1'].append(val_metrics['f1_score'])
        
#         print(f"Train | Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
#         print(f"Val   | Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_score']:.4f}")
        
#         scheduler.step()

#         if val_metrics['accuracy'] > best_val_acc:
#             best_val_acc = val_metrics['accuracy']
#             best_val_metrics = val_metrics
#             torch.save(model.state_dict(), os.path.join(current_output_dir, 'best_model_adl.pth'))
#             print(f"New best model for '{class_name}' saved with val acc: {best_val_acc:.4f}")

#     end_time_pk = datetime.now(pk_timezone)
#     results[class_name] = best_val_acc
#     print(f"\n--- Training for '{class_name}' Finished ---")
#     print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
#     print(f"Best validation accuracy for '{class_name}': {best_val_acc:.4f}")
    
#     # --- Save Metrics and Generate Plots ---
    
#     # Create a DataFrame from the history
   
# # --- Final Summary ---
# print("\n========================================================")
# print("              ALL TRAINING RUNS COMPLETE")
# print("========================================================")
# for class_name, acc in results.items():
#     print(f"  - Best validation accuracy for '{class_name}': {acc:.4f}")
# print("========================================================")
# for class_name in Config.CLASS_NAMES:
#     print(f"\n========================================================")
#     print(f"        STARTING TRAINING FOR CLASS: {class_name}")
#     print(f"========================================================")

#     # --- Paths and Directory Setup ---
#     current_data_dir = os.path.join(Config.base_data_dir, class_name)
#     current_output_dir = os.path.join(Config.base_output_dir, class_name)
#     os.makedirs(current_output_dir, exist_ok=True)
    
#     # --- Data Loading ---
#     train_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'train', 'train_data.pkl'))
#     val_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'val', 'val_data.pkl'))
#     train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, num_workers=Config.num_workers)
#     val_loader = DataLoader(val_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=Config.num_workers)

#     # --- Model, Loss, Optimizer ---
#     model = ADLModel(Config.num_classes, Config.drop_rate, Config.drop_threshold).to(device)
#     criterion_cls = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=Config.lr)
#     scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)

#     # --- Checkpoint Loading Logic ---
#     checkpoint_path = os.path.join(current_output_dir, 'checkpoint.pth')
#     start_epoch = 0
#     best_val_acc = 0.0
#     history = {
#         'train_loss': [], 'train_acc': [],
#         'val_loss': [], 'val_acc': [],
#         'val_precision': [], 'val_recall': [], 'val_f1': []
#     }

#     if os.path.exists(checkpoint_path):
#         print(f"✅ Resuming training for '{class_name}' from checkpoint.")
#         # Load checkpoint onto the correct device
#         # Tell torch.load it's okay to unpickle other Python objects besides tensors
#         checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
#         model.load_state_dict(checkpoint['model_state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
#         start_epoch = checkpoint['epoch'] + 1
#         history = checkpoint['history']
#         best_val_acc = checkpoint['best_val_acc']
#         print(f"   Resumed from epoch {start_epoch}. Best validation accuracy so far: {best_val_acc:.4f}")
#     else:
#         print(f"ℹ️ No checkpoint found for '{class_name}'. Starting training from scratch.")


#     # --- Training Loop ---
#     best_val_metrics = None
#     start_time_pk = datetime.now(pk_timezone)
#     print(f"Start Time (Lahore): {start_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")

#     # **MODIFIED**: Start the loop from 'start_epoch'
#     for epoch in range(start_epoch, Config.epochs):
#         print(f"\n--- Epoch {epoch+1}/{Config.epochs} ---")
        
#         train_metrics = train_one_epoch(model, train_loader, criterion_cls, optimizer, device, Config.adl_alpha)
#         val_metrics = validate(model, val_loader, criterion_cls, device)
        
#         # Log metrics
#         history['train_loss'].append(train_metrics['loss'])
#         history['train_acc'].append(train_metrics['accuracy'])
#         history['val_loss'].append(val_metrics['loss'])
#         history['val_acc'].append(val_metrics['accuracy'])
#         history['val_precision'].append(val_metrics['precision'])
#         history['val_recall'].append(val_metrics['recall'])
#         history['val_f1'].append(val_metrics['f1_score'])
        
#         print(f"Train | Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
#         print(f"Val   | Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_score']:.4f}")
        
#         scheduler.step()

#         if val_metrics['accuracy'] > best_val_acc:
#             best_val_acc = val_metrics['accuracy']
#             best_val_metrics = val_metrics
#             torch.save(model.state_dict(), os.path.join(current_output_dir, 'best_model_adl.pth'))
#             print(f"New best model for '{class_name}' saved with val acc: {best_val_acc:.4f}")

#         # --- Save Checkpoint After Each Epoch ---
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'scheduler_state_dict': scheduler.state_dict(),
#             'best_val_acc': best_val_acc,
#             'history': history,
#         }, checkpoint_path)

#     end_time_pk = datetime.now(pk_timezone)
#     results[class_name] = best_val_acc
#     print(f"\n--- Training for '{class_name}' Finished ---")
#     print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
#     print(f"Best validation accuracy for '{class_name}': {best_val_acc:.4f}")
    
#     # --- Save Metrics and Generate Plots ---
#     # Create a DataFrame from the history
#     # ... (your existing code for plotting and saving metrics) ...
    
# # --- Final Summary ---
# print("\n========================================================")
# print("               ALL TRAINING RUNS COMPLETE")
# print("========================================================")
# for class_name, acc in results.items():
#     print(f"  - Best validation accuracy for '{class_name}': {acc:.4f}")
# print("========================================================")

# A list to store the final F1 score (or accuracy) from each fold
all_fold_results = []

for class_name in Config.CLASS_NAMES:
    print(f"\n========================================================")
    print(f"       STARTING {Config.N_FOLDS}-FOLD CV FOR CLASS: {class_name}")
    print(f"========================================================")
    
    class_data_dir = os.path.join(Config.base_data_dir, class_name)
    class_output_dir = os.path.join(Config.base_output_dir, class_name)

    # --- NEW: Outer loop for each fold ---
    for fold_idx in range(1, Config.N_FOLDS + 1):
        print(f"\n--- Starting Fold {fold_idx}/{Config.N_FOLDS} ---")

        # --- MODIFIED: Paths now point to the specific fold's data and output ---
        current_data_dir = os.path.join(class_data_dir, f'fold_{fold_idx}')
        current_output_dir = os.path.join(class_output_dir, f'fold_{fold_idx}')
        os.makedirs(current_output_dir, exist_ok=True)
        
        # --- Data Loading for the current fold ---
        train_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'train_data.pkl'))
        val_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'val_data.pkl'))
        train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=Config.batch_size, shuffle=False)

        # --- IMPORTANT: Re-initialize model and optimizer for each fold ---
        model = ADLModel(Config.num_classes, Config.drop_rate, Config.drop_threshold).to(device)
        optimizer = optim.Adam(model.parameters(), lr=Config.lr)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)

        # --- Checkpointing logic (as you wrote it) would go here ---
        # It will now save/load checkpoints inside the fold-specific output directory
        checkpoint_path = os.path.join(current_output_dir, 'checkpoint.pth')
        # ... (your checkpoint loading code)
       
        start_epoch = 0
        best_val_acc = 0.0
        history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [],
            'val_precision': [], 'val_recall': [], 'val_f1': []
        }
    
        if os.path.exists(checkpoint_path):
            print(f"✅ Resuming training for '{class_name}' from checkpoint.")
            # Load checkpoint onto the correct device
            # Tell torch.load it's okay to unpickle other Python objects besides tensors
            checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            history = checkpoint['history']
            best_val_acc = checkpoint['best_val_acc']
            print(f"   Resumed from epoch {start_epoch}. Best validation accuracy so far: {best_val_acc:.4f}")
        else:
            print(f"ℹ️ No checkpoint found for '{class_name}'. Starting training from scratch.")
    


        # --- Training Loop (your existing code for epochs) ---
        best_val_f1 = 0.0 # Or best_val_acc
        for epoch in range(start_epoch, Config.epochs):
            print(f"\n--- Epoch {epoch+1}/{Config.epochs} ---")
            
            train_metrics = train_one_epoch(model, train_loader, criterion_cls, optimizer, device, Config.adl_alpha)
            val_metrics = validate(model, val_loader, criterion_cls, device)
            
            # Log metrics
            history['train_loss'].append(train_metrics['loss'])
            history['train_acc'].append(train_metrics['accuracy'])
            history['val_loss'].append(val_metrics['loss'])
            history['val_acc'].append(val_metrics['accuracy'])
            history['val_precision'].append(val_metrics['precision'])
            history['val_recall'].append(val_metrics['recall'])
            history['val_f1'].append(val_metrics['f1_score'])
            
            print(f"Train | Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
            print(f"Val   | Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_score']:.4f}")
            
            scheduler.step()
    
            if val_metrics['accuracy'] > best_val_acc:
                best_val_acc = val_metrics['accuracy']
                best_val_metrics = val_metrics
                torch.save(model.state_dict(), os.path.join(current_output_dir, 'best_model_adl.pth'))
                print(f"New best model for '{class_name}' saved with val acc: {best_val_acc:.4f}")
    
            # --- Save Checkpoint After Each Epoch ---
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_acc': best_val_acc,
                'history': history,
            }, checkpoint_path)
    
    end_time_pk = datetime.now(pk_timezone)
    results[class_name] = best_val_acc
    print(f"\n--- Training for '{class_name}' Finished ---")
    print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
    print(f"Best validation accuracy for '{class_name}': {best_val_acc:.4f}")
        
        # --- Save Metrics and Generate Plots ---
        # Create a DataFrame from the history
        # ... (your existing code for plotting and saving metrics) ...
        
    # --- Final Summary ---
    print("\n========================================================")
    print("               ALL TRAINING RUNS COMPLETE")
    print("========================================================")
    for class_name, acc in results.items():
        print(f"  - Best validation accuracy for '{class_name}': {acc:.4f}")
    print("========================================================")
           # ... (your for loop over epochs)
           #     ... (your training and validation calls)
            #    ... (save best model to current_output_dir)
        
        # --- Store the best result for this fold ---
    print(f"--- Best F1 Score for Fold {fold_idx}: {best_val_f1:.4f} ---")
    all_fold_results.append(best_val_f1)

    # --- Aggregate and report the final average score for the class ---
    average_score = sum(all_fold_results) / len(all_fold_results)
    print(f"\n--- Average Cross-Validation F1 Score for '{class_name}': {average_score:.4f} ---")
    all_fold_results.clear() # Reset for the next class

Using device: cuda

       STARTING 3-FOLD CV FOR CLASS: lighting_panel

--- Starting Fold 1/3 ---
Loading data from processed_data_multi_class_3_fold/lighting_panel/fold_1/train_data.pkl...
Data loaded successfully.
Loading data from processed_data_multi_class_3_fold/lighting_panel/fold_1/val_data.pkl...
Data loaded successfully.
ℹ️ No checkpoint found for 'lighting_panel'. Starting training from scratch.

--- Epoch 1/50 ---


NameError: name 'criterion_cls' is not defined

In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datetime import datetime
import pytz
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# --- Assume these are defined elsewhere ---
# from config import config 
# from model import ADLModel
# from dataset import PreprocessedDataset
# from training_utils import train_one_epoch, validate, plot_metrics
# pk_timezone = pytz.timezone("Asia/Karachi")

def run_cross_validation_experiment():
    """
    Main function to run the entire training and 3-fold cross-validation process.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Store the final averaged results for each class
    final_class_results = {}

    for class_name in config.CLASS_NAMES:
        print(f"\n========================================================")
        print(f"    STARTING {config.N_FOLDS}-FOLD CV FOR CLASS: {class_name}")
        print(f"========================================================")
        
        class_data_dir = os.path.join(config.base_data_dir, class_name)
        class_output_dir = os.path.join(config.base_output_dir, class_name)
        
        # Store the best F1 score from each fold for the current class
        fold_performance_scores = []

        # --- NEW: Outer loop for iterating through each fold ---
        for fold_idx in range(1, config.N_FOLDS + 1):
            print(f"\n--- Starting Fold {fold_idx}/{config.N_FOLDS} ---")

            # --- MODIFIED: Paths now point to the specific fold's data and output ---
            current_data_dir = os.path.join(class_data_dir, f'fold_{fold_idx}')
            current_output_dir = os.path.join(class_output_dir, f'fold_{fold_idx}')
            os.makedirs(current_output_dir, exist_ok=True)
            
            # --- Data Loading for the current fold ---
            train_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'train_data.pkl'))
            val_dataset = PreprocessedDataset(os.path.join(current_data_dir, 'val_data.pkl'))
            train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
            val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

            # --- IMPORTANT: Re-initialize model and optimizer for each fold ---
            model = ADLModel(config.num_classes, config.drop_rate, config.drop_threshold).to(device)
            optimizer = optim.Adam(model.parameters(), lr=config.lr)
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45], gamma=0.1)
            criterion_cls = nn.CrossEntropyLoss()

            # --- Checkpoint Loading Logic (now fold-specific) ---
            checkpoint_path = os.path.join(current_output_dir, 'checkpoint.pth')
            start_epoch = 0
            best_val_f1 = 0.0 # Using F1 score as the key metric
            history = {
                'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [],
                'val_precision': [], 'val_recall': [], 'val_f1': []
            }

            if os.path.exists(checkpoint_path):
                print(f"✅ Resuming training for fold {fold_idx} from checkpoint.")
                checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
                history = checkpoint['history']
                best_val_f1 = checkpoint['best_val_metric']
                print(f"   Resumed from epoch {start_epoch}. Best F1 so far: {best_val_f1:.4f}")
            else:
                print(f"ℹ️ No checkpoint found for fold {fold_idx}. Starting from scratch.")

            # --- Training Loop for Epochs ---
            start_time_pk = datetime.now(pk_timezone)
            print(f"Start Time (Lahore): {start_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")

            for epoch in range(start_epoch, config.epochs):
                print(f"\n--- Epoch {epoch+1}/{config.epochs} ---")
                
                train_metrics = train_one_epoch(model, train_loader, criterion_cls, optimizer, device, config.adl_alpha)
                val_metrics = validate(model, val_loader, criterion_cls, device)
                
                # Log metrics to history
                history['train_loss'].append(train_metrics['loss'])
                history['train_acc'].append(train_metrics['accuracy'])
                history['val_loss'].append(val_metrics['loss'])
                history['val_acc'].append(val_metrics['accuracy'])
                history['val_precision'].append(val_metrics['precision'])
                history['val_recall'].append(val_metrics['recall'])
                history['val_f1'].append(val_metrics['f1_score'])
                
                print(f"Train | Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
                print(f"Val   | Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_score']:.4f}")
                
                scheduler.step()

                # Save the best model based on F1 score
                if val_metrics['f1_score'] > best_val_f1:
                    best_val_f1 = val_metrics['f1_score']
                    torch.save(model.state_dict(), os.path.join(current_output_dir, 'best_model.pth'))
                    print(f"New best model saved for fold {fold_idx} with F1 score: {best_val_f1:.4f}")

                # Save checkpoint after each epoch
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_metric': best_val_f1,
                    'history': history,
                }, checkpoint_path)

            end_time_pk = datetime.now(pk_timezone)
            print(f"\n--- Training for Fold {fold_idx} Finished ---")
            print(f"End Time (Lahore): {end_time_pk.strftime('%Y-%m-%d %I:%M:%S %p')}")
            print(f"Best validation F1 for this fold: {best_val_f1:.4f}")
            
            # --- Store the best result for this fold ---
            fold_performance_scores.append(best_val_f1)
            
            # --- Save Metrics and Plots for the fold ---
            # You can add your plotting and metrics saving logic here
            # Example: plot_metrics(history, current_output_dir)

        # --- Aggregate and report the final average score for the class ---
        if fold_performance_scores:
            average_score = sum(fold_performance_scores) / len(fold_performance_scores)
            final_class_results[class_name] = average_score
            print(f"\n--- Average Cross-Validation F1 Score for '{class_name}': {average_score:.4f} ---")
        else:
            print(f"\n--- No folds were successfully trained for '{class_name}' ---")


    # --- Final Summary ---
    print("\n========================================================")
    print("           ALL CROSS-VALIDATION RUNS COMPLETE")
    print("========================================================")
    for class_name, avg_score in final_class_results.items():
        print(f"  - Average F1 Score for '{class_name}': {avg_score:.4f}")
    print("========================================================")

if __name__ == '__main__':
    # Ensure you have a 'config' object available, loaded from your Config class.
    run_cross_validation_experiment()
    

Using device: cuda

    STARTING 3-FOLD CV FOR CLASS: lighting_panel

--- Starting Fold 1/3 ---
Loading data from processed_data_multi_class_3_fold/lighting_panel/fold_1/train_data.pkl...
Data loaded successfully.
Loading data from processed_data_multi_class_3_fold/lighting_panel/fold_1/val_data.pkl...
Data loaded successfully.
ℹ️ No checkpoint found for fold 1. Starting from scratch.
Start Time (Lahore): 2025-10-10 11:56:44 AM

--- Epoch 1/50 ---
Train | Loss: 0.7263, Acc: 0.5376
Val   | Loss: 0.6802, Acc: 0.5857, F1: 0.0000

--- Epoch 2/50 ---
Train | Loss: 0.6883, Acc: 0.5680
Val   | Loss: 0.6816, Acc: 0.5857, F1: 0.0000

--- Epoch 3/50 ---
Train | Loss: 0.6796, Acc: 0.5824
Val   | Loss: 0.6725, Acc: 0.5857, F1: 0.0000

--- Epoch 4/50 ---
Train | Loss: 0.6797, Acc: 0.5558
Val   | Loss: 0.6903, Acc: 0.4416, F1: 0.5974
New best model saved for fold 1 with F1 score: 0.5974

--- Epoch 5/50 ---
Train | Loss: 0.6813, Acc: 0.5953
Val   | Loss: 0.6707, Acc: 0.5857, F1: 0.0000

--- Epoch 6/5

In [16]:
import os
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report
import pandas as pd
  
history = {
    'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'val_precision': [], 'val_recall': [], 'val_f1': []
    }
history_df = pd.DataFrame(history)
history_df.index.name = 'epoch'
    
    # Save the DataFrame to a CSV file
csv_path = os.path.join(current_output_dir, f'{class_name}_training_history.csv')
history_df.to_csv(csv_path)
print(f"\n--- Metrics for '{class_name}' saved to {csv_path} ---")

print(f"\n--- Generating plots for {class_name} ---")
# plot_metrics(history, class_name, current_output_dir)
# if best_val_metrics:
#         plot_confusion_matrix(best_val_metrics['labels'], best_val_metrics['predictions'], class_name, current_output_dir)
#         print("\n--- Final Classification Report (Best Model) ---")
#         print(classification_report(best_val_metrics['labels'], best_val_metrics['predictions'], target_names=['non-defect', 'defect']))




--- Metrics for 'frosted_window' saved to trained_models_multi_class/frosted_window/fold_3/frosted_window_training_history.csv ---

--- Generating plots for frosted_window ---
